In [1]:
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import collections
from annoy import AnnoyIndex
import os
import copy

In [2]:
debug=False
bucketSize=20
minValueOfNrHitsForParticleWithMostHits=0

In [3]:
# pt orice proiect fisier de input din care sa iau date si fisier de output in care sa pun rezultatul obtinut
inputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/data/TrackML/ttbar_mu200-generic"
outputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackML/outputC2"

In [4]:
def get_eventNumber_from_fileName(fileName):
    eventNumber=fileName.replace("event","").replace("-hits.csv","")
    return eventNumber

In [5]:
# calculate event numbers from my folder
# lista goala in care sa pun numerele evenimentelor
list_eventNumber=[]
# The sorted() function returns a sorted list of the specified iterable object 
# Strings are sorted alphabetically, and numbers are sorted numerically
# os operating system 
# hits ca sa nu am de doura ori evenimentul in lista
for fileName in sorted(os.listdir(inputFolderName)):
    if fileName.endswith("-hits.csv"):
        #print(fileName)
        eventNumber=get_eventNumber_from_fileName(fileName)
        #print(eventNumber)
        list_eventNumber.append(eventNumber)
# done for loop
#list_eventNumber=["000000007"]
print("All events available in my folder. list_eventNumber", list_eventNumber)
    

All events available in my folder. list_eventNumber ['000000007']


In [6]:
# this runs only once per event
def buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False): 
    
    numberDimension=nparray_position.shape[1] # 3 (x,y,z)
    if debug:
        print("numberDimension",numberDimension,"metric",metric)
    index=AnnoyIndex(numberDimension,metric)
    if debug:
        print("type(index)",type(index))
        print("enumerate data")
    # add each hit to the index
    for i,position in enumerate(nparray_position):
        if debug:
            print("i",i,"position",position)
        index.add_item(i,position)
    # done for loop over hits
    # build the index with 10 trees
    index.build(ntrees) # 10 trees
    return index
# done function

In [7]:
def get_input_output_from_df_hits(df_hits,index):
    list_nparray_input=[]
    list_nparray_output=[]
    
    nparray_volume_id=df_hits["volume_id"].values
    nparray_layer_id=df_hits["layer_id"].values
    
    for i in range(len(df_hits)):
        if (nparray_volume_id[i]==8 and nparray_layer_id[i]==2)==False:
            continue
            
        # using annoy to find the 20 nearest neighboring hits by angle to this hit
        list_index=index.get_nns_by_item(i,bucketSize)
        if debug:
            print("list_index",list_index)
            
        
            
        # create bucket
        df_bucket=df_hits.iloc[list_index]
        
        # create NN input for this one bucket
        nparray_input=df_bucket[["x","y","z"]].values.flatten()
        if debug:
            print("nparray_input",nparray_input,"shape",nparray_input.shape,"type",type(nparray_input))
            
        
            
        # create NN output for this one bucket
        # identify particle with the largest number of hits in this bucket
        nparray_particleID=df_bucket["particle_id"].values
        

        if debug:
            print("nparray_particleID",nparray_particleID,"shape",nparray_particleID.shape,"type",type(nparray_particleID))
        dict_particleID_counterParticleID={}
        for particleID in nparray_particleID:
            if particleID not in dict_particleID_counterParticleID:
                dict_particleID_counterParticleID[particleID]=1
            else:
                dict_particleID_counterParticleID[particleID]+=1
        if debug:
            print("dict_particleID_counterParticleID",dict_particleID_counterParticleID)
          
        # find the maximum value of the counters
        particleIDWithMaxHits=0
        counterParticleIDWithMaxHits=0
        for particleID in dict_particleID_counterParticleID:
            counterParticleID=dict_particleID_counterParticleID[particleID]
            if counterParticleID>counterParticleIDWithMaxHits:
                counterParticleIDWithMaxHits=counterParticleID
                particleIDWithMaxHits=particleID
        if debug:
            print("counterBucket",counterBucket,"particleIDWithMaxHits",particleIDWithMaxHits,"counterParticleIDWithMaxHits",counterParticleIDWithMaxHits)        
        
        # create nparray_output
        list_output=[]
        
        # loop over every hit in the bucket
        for particleID in nparray_particleID:      
            if counterParticleIDWithMaxHits<minValueOfNrHitsForParticleWithMostHits:
                list_output.append(-1)
            else:
                # do normally
                if particleID==particleIDWithMaxHits:
                    list_output.append(1)
                else:
                    list_output.append(-1)
                # done if
            # done if
        # done for loop for each hit in the bucket
        if debug:        
            print("list_output",list_output)
        nparray_output=np.array(list_output)
        if debug:
            print("nparray_output",nparray_output)
            
        if debug and doTest:
            print("i",i,"counterBucket",counterBucket)
            plt.plot(df_bucket.x,df_bucket.y,"o",color="red")
            plt.plot(0,0,"r+")
            plt.title(str(i))
            plt.show()
            plotBucketFileNameStem=outputFolderName+"/bucket_i_"+str(i)+"_counterBucket_"+str(counterBucket)+"_eventNumber_"+eventNumber
            # plt.savefig(plotBucketFileNameStem+".png")
            # plt.savefig(plotBucketFileNameStem+".pdf")
        # done if
            
        # add for this current bucket to the list for the entire event (which has several buckets)
        list_nparray_input.append(nparray_input)
        list_nparray_output.append(nparray_output)
    # done for loop over hits in the event
    
    # transform list of numpy array into numpy array of dimension 2 
    # rows are buckets, columns are intput and output values
    nparray_input_all=np.array(list_nparray_input)
    nparray_output_all=np.array(list_nparray_output)
    
    return nparray_input_all, nparray_output_all
# done function      

In [8]:
# for loop over eventNumber from list_eventNumber
for i,eventNumber in enumerate(list_eventNumber):
    print("i",i,"eventNumber",eventNumber)
    
    inputFileName_hits_recon=inputFolderName+"/event"+eventNumber+"-hits.csv"
    inputFileName_hits_truth=inputFolderName+"/event"+eventNumber+"-truth.csv"
    
    df_hits_recon=pd.read_csv(inputFileName_hits_recon)
    df_hits_truth=pd.read_csv(inputFileName_hits_truth)
    df_hits=pd.concat([df_hits_recon,df_hits_truth],axis=1,sort=False)
    
    # build annoy index
    nparray_position=df_hits[["x","y","z"]].values
    if debug:
        print("nparray_position",nparray_position)

    index=buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False)
    
    # for loop peste hituri peste hiturile preselectate volume_id=8, layer_id=2
    nparray_input_all,nparray_output_all=get_input_output_from_df_hits(df_hits,index)
    if debug:
        print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all)) 
        print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))

    # spliting part
    # insure that the number of buckets is even, si to split later in train and test
    nrBucket=nparray_input_all.shape[0]
    if  nrBucket%2==1:
        nparray_input_all=nparray_input_all[0:-1]
        nparray_output_all=nparray_output_all[0:-1]
    else:
        pass
    if debug:
        print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
        print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))
        
    # reshape only input by adding one extra dimension needed by tensorflow, in practice it puts all into one extra bracket
    nparray_input_all=nparray_input_all.reshape(nparray_input_all.shape[0],nparray_input_all.shape[1],1)
    if debug:
        print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))

    # spliting; list comprehension
    nrBucket=nparray_input_all.shape[0]
    list_index_Train=[i for i in range(nrBucket) if i%2==0]
    list_index_Test=[i for i in range(nrBucket) if i%2==1]

    nparray_Input_Train=nparray_input_all[list_index_Train]
    nparray_Input_Test=nparray_input_all[list_index_Test]
    nparray_Output_Train=nparray_output_all[list_index_Train]
    nparray_Output_Test=nparray_output_all[list_index_Test]
    
    if i==0:
        nparray_Input_Train_all_Events=copy.deepcopy(nparray_Input_Train)
        nparray_Input_Test_all_Events=copy.deepcopy(nparray_Input_Test)
        nparray_Output_Train_all_Events=copy.deepcopy(nparray_Output_Train)
        nparray_Output_Test_all_Events=copy.deepcopy(nparray_Output_Test)
    else:
        nparray_Input_Train_all_Events=np.concatenate((nparray_Input_Train_all_Events,nparray_Input_Train),axis=0,out=None)
        nparray_Input_Test_all_Events=np.concatenate((nparray_Input_Test_all_Events,nparray_Input_Test),axis=0,out=None)
        nparray_Output_Train_all_Events=np.concatenate((nparray_Output_Train_all_Events,nparray_Output_Train),axis=0,out=None)
        nparray_Output_Test_all_Events=np.concatenate((nparray_Output_Test_all_Events,nparray_Output_Test),axis=0,out=None)
    # done if
# done for loop over eventNumber

eventNumber="all"
fileNameNNInputTrainAll=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
fileNameNNInputTestAll=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
fileNameNNOutputTrainAll=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"
fileNameNNOutputTestAll=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"

np.save(fileNameNNInputTrainAll,nparray_Input_Train_all_Events)
np.save(fileNameNNOutputTrainAll,nparray_Output_Train_all_Events)
np.save(fileNameNNInputTestAll,nparray_Input_Test_all_Events)
np.save(fileNameNNOutputTestAll,nparray_Output_Test_all_Events)

print("nparray_Input_Train_all_Events",nparray_Input_Train_all_Events,"shape",nparray_Input_Train_all_Events.shape,"type",type(nparray_Input_Train_all_Events))
print("nparray_Input_Test_all_Events",nparray_Input_Test_all_Events,"shape",nparray_Input_Test_all_Events.shape,"type",type(nparray_Input_Test_all_Events))
print("nparray_Output_Train_all_Events",nparray_Output_Train_all_Events,"shape",nparray_Output_Train_all_Events.shape,"type",type(nparray_Output_Train_all_Events))
print("nparray_Output_Test_all_Events",nparray_Output_Test_all_Events,"shape",nparray_Output_Test_all_Events.shape,"type",type(nparray_Output_Test_all_Events))

print("Done all!")

i 0 eventNumber 000000007
nparray_Input_Train_all_Events [[[-3.27351e+01]
  [-3.13191e+00]
  [-4.42867e+02]
  ...
  [-4.44534e+01]
  [-1.76822e+00]
  [-6.02500e+02]]

 [[-3.05373e+01]
  [-9.41792e+00]
  [-4.80159e+02]
  ...
  [-4.29130e+01]
  [-1.39791e+01]
  [-6.98000e+02]]

 [[-3.36008e+01]
  [-6.55743e-01]
  [-4.51391e+02]
  ...
  [-6.22143e+01]
  [-1.99989e+00]
  [-8.17500e+02]]

 ...

 [[-3.09951e+01]
  [ 8.76215e+00]
  [ 4.39276e+02]
  ...
  [-5.03296e+01]
  [ 1.54623e+01]
  [ 6.98000e+02]]

 [[-3.09529e+01]
  [ 9.50895e+00]
  [ 4.26645e+02]
  ...
  [-8.28716e+01]
  [ 2.29552e+01]
  [ 1.10200e+03]]

 [[-2.87558e+01]
  [ 1.27489e+01]
  [ 4.53941e+02]
  ...
  [-3.63047e+01]
  [ 1.46320e+01]
  [ 5.98000e+02]]] shape (5458, 60, 1) type <class 'numpy.ndarray'>
nparray_Input_Test_all_Events [[[-3.72866e+01]
  [-1.21141e+01]
  [-5.98000e+02]
  ...
  [-4.40492e+01]
  [-1.35834e+01]
  [-6.98000e+02]]

 [[-3.31042e+01]
  [-2.07623e+00]
  [-4.35250e+02]
  ...
  [-3.20580e+01]
  [-1.28937e+0